In [37]:
from data_provider.data_factory import data_provider
from exp.exp_basic import Exp_Basic
from utils.tools import EarlyStopping, adjust_learning_rate, cal_accuracy
import torch
import torch.nn as nn
from torch import optim
import os
import time
import warnings
import pandas as pd
import numpy as np
from models import Autoformer, Transformer, TimesNet, Nonstationary_Transformer, DLinear, FEDformer, \
    Informer, LightTS, Reformer, ETSformer, Pyraformer, PatchTST, MICN, Crossformer, FiLM
from data_provider.uea import subsample, interpolate_missing, Normalizer

In [38]:
class arg():
    task_name = 'classification'
    is_training = 1
    model_id = 'test'
    model = 'Autoformer'
    data = 'ETTm1'
    root_path = './data/ETT/'
    data_path = 'ETTh1.csv'
    features = 'M'
    target = 'OT'
    freq = 'h'
    checkpoints = './checkpoints/'
    seq_len = 96
    label_len = 48
    pred_len = 96
    seasonal_patterns = 'Monthly'
    mask_rate = 0.25
    anomaly_ratio = 0.25
    top_k = 5
    num_kernels = 6
    enc_in = 7
    dec_in = 7
    c_out = 7
    d_model = 512
    n_heads = 8
    e_layers = 2
    d_layers = 1
    d_ff = 2048
    moving_avg = 25
    factor = 1
    distil = True
    dropout = 0.1
    embed = 'timeF'
    activation = 'gelu'
    output_attention = True
    num_workers = 10
    itr = 1
    train_epochs = 10
    batch_size = 32
    patience = 3
    learning_rate = 0.0001
    des = 'test'
    loss = 'MSE'
    lradj = 'type1'
    use_amp = False
    use_gpu = True
    gpu = 0
    use_multi_gpu = False
    devices = '0,1,2,3'
    p_hidden_dims = [128, 128]
    p_hidden_layers = 2

args = arg()
args.task_name = 'classification'
args.is_training = 1
args.root_path = './dataset/Huawei/'
args.model_id = 'Huawei'
args.model = 'TimesNet'
args.data = 'Huawei'
args.e_layers = 2
args.batch_size = 16
args.d_model = 64
args.d_ff = 64
args.top_k = 1
args.des = 'Exp'
args.itr = 1
args.learning_rate = 0.001
args.train_epochs = 30
args.patience = 10
args.use_gpu = False


In [39]:
def get_data(flag):
    data_set, data_loader = data_provider(args, flag)
    return data_set, data_loader

def select_optimizer():
    model_optim = optim.Adam(model.parameters(), lr=args.learning_rate)
    return model_optim

def select_criterion():
    criterion = nn.CrossEntropyLoss()
    return criterion

model_dict = {
            'TimesNet': TimesNet,
            'Autoformer': Autoformer,
            'Transformer': Transformer,
            'Nonstationary_Transformer': Nonstationary_Transformer,
            'DLinear': DLinear,
            'FEDformer': FEDformer,
            'Informer': Informer,
            'LightTS': LightTS,
            'Reformer': Reformer,
            'ETSformer': ETSformer,
            'PatchTST': PatchTST,
            'Pyraformer': Pyraformer,
            'MICN': MICN,
            'Crossformer': Crossformer,
            'FiLM': FiLM,
        }
def build_model():
    # model input depends on data
    train_data, train_loader = get_data(flag='TRAIN')
    test_data, test_loader = get_data(flag='TEST')
    args.seq_len = max(train_data.max_seq_len, test_data.max_seq_len)
    args.pred_len = 0
    args.enc_in = train_data.feature_df.shape[1]
    args.num_class = len(train_data.class_names)
    # model init
    model = model_dict[args.model].Model(args).float()
    if args.use_multi_gpu and args.use_gpu:
        model = nn.DataParallel(model, device_ids=args.device_ids)
    return model

def acquire_device():
    if args.use_gpu:
        os.environ["CUDA_VISIBLE_DEVICES"] = str(
            args.gpu) if not args.use_multi_gpu else args.devices
        device = torch.device('cuda:{}'.format(args.gpu))
        print('Use GPU: cuda:{}'.format(args.gpu))
    else:
        device = torch.device('cpu')
        print('Use CPU')
    return device

def load_data(filepath):
    df = pd.read_pickle(filepath)
    df = df.reset_index()
    df['label'] = df['抑郁得分'].apply(lambda x: 'depression' if x > 4 else 'non-depression')
    labels = df['label'].values
    df = df[['heartrate', 'bloodoxygen', 'step']]
    labels = pd.Series(labels, dtype="category")
    class_names = labels.cat.categories
    labels_df = pd.DataFrame(labels.cat.codes,
                                dtype=np.int8)  # int8-32 gives an error when using nn.CrossEntropyLoss

    lengths = df.applymap(
        lambda x: len(x)).values  # (num_samples, num_dimensions) array containing the length of each series

    horiz_diffs = np.abs(lengths - np.expand_dims(lengths[:, 0], -1))

    if np.sum(horiz_diffs) > 0:  # if any row (sample) has varying length across dimensions
        df = df.applymap(subsample)

    lengths = df.applymap(lambda x: len(x)).values
    vert_diffs = np.abs(lengths - np.expand_dims(lengths[0, :], 0))
    if np.sum(vert_diffs) > 0:  # if any column (dimension) has varying length across samples
        max_seq_len = int(np.max(lengths[:, 0]))
    else:
        max_seq_len = lengths[0, 0]

    # First create a (seq_len, feat_dim) dataframe for each sample, indexed by a single integer ("ID" of the sample)
    # Then concatenate into a (num_samples * seq_len, feat_dim) dataframe, with multiple rows corresponding to the
    # sample index (i.e. the same scheme as all datasets in this project)

    # df = pd.concat((pd.DataFrame({col: df.loc[row, col] for col in df.columns}).reset_index(drop=True).set_index(
    #     pd.Series(lengths[row, 0] * [row])) for row in range(df.shape[0])), axis=0)
    
    a_list = []

    for row in range(df.shape[0]):
        a = pd.DataFrame({col: df.loc[row, col] for col in df.columns}).reset_index(drop=True)
        a = a.set_index(pd.Series(lengths[row, 0] * [row]))
        a_list.append(a)

    df = pd.concat(a_list, axis=0)

    # Replace NaN values
    grp = df.groupby(by=df.index)
    df = grp.transform(interpolate_missing)

    return df, labels_df

In [40]:
setting = '{}_{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(
    args.task_name,
    args.model_id,
    args.model,
    args.data,
    args.features,
    args.seq_len,
    args.label_len,
    args.pred_len,
    args.d_model,
    args.n_heads,
    args.e_layers,
    args.d_layers,
    args.d_ff,
    args.factor,
    args.embed,
    args.distil,
    args.des, 0)
model = build_model()
device = acquire_device()


40
11
Use CPU


In [41]:
def vali(vali_data, vali_loader, criterion):
    total_loss = []
    preds = []
    trues = []
    model.eval()
    with torch.no_grad():
        for i, (batch_x, label, padding_mask) in enumerate(vali_loader):
            batch_x = batch_x.float().to(device)
            padding_mask = padding_mask.float().to(device)
            label = label.to(device)

            outputs = model(batch_x, padding_mask, None, None)

            pred = outputs.detach().cpu()
            loss = criterion(pred, label.long().squeeze().cpu())
            total_loss.append(loss)

            preds.append(outputs.detach())
            trues.append(label)

    total_loss = np.average(total_loss)

    preds = torch.cat(preds, 0)
    trues = torch.cat(trues, 0)
    probs = torch.nn.functional.softmax(preds)  # (total_samples, num_classes) est. prob. for each class and sample
    predictions = torch.argmax(probs, dim=1).cpu().numpy()  # (total_samples,) int class index for each sample
    trues = trues.flatten().cpu().numpy()
    accuracy = cal_accuracy(predictions, trues)

    model.train()
    return total_loss, accuracy

In [42]:
def train():
    train_data, train_loader = get_data(flag='TRAIN')
    vali_data, vali_loader = get_data(flag='TRAIN')
    test_data, test_loader = get_data(flag='TRAIN')

    path = os.path.join(args.checkpoints, setting)
    if not os.path.exists(path):
        os.makedirs(path)

    time_now = time.time()

    train_steps = len(train_loader)
    early_stopping = EarlyStopping(patience=args.patience, verbose=True)

    model_optim = select_optimizer()
    criterion = select_criterion()

    for epoch in range(args.train_epochs):
        iter_count = 0
        train_loss = []

        model.train()
        epoch_time = time.time()

        for i, (batch_x, label, padding_mask) in enumerate(train_loader):
            iter_count += 1
            model_optim.zero_grad()

            batch_x = batch_x.float().to(device)
            padding_mask = padding_mask.float().to(device)
            label = label.to(device)

            outputs = model(batch_x, padding_mask, None, None)
            loss = criterion(outputs, label.long().squeeze(-1))
            train_loss.append(loss.item())

            if (i + 1) % 100 == 0:
                print("\titers: {0}, epoch: {1} | loss: {2:.7f}".format(i + 1, epoch + 1, loss.item()))
                speed = (time.time() - time_now) / iter_count
                left_time = speed * ((args.train_epochs - epoch) * train_steps - i)
                print('\tspeed: {:.4f}s/iter; left time: {:.4f}s'.format(speed, left_time))
                iter_count = 0
                time_now = time.time()

            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=4.0)
            model_optim.step()

        print("Epoch: {} cost time: {}".format(epoch + 1, time.time() - epoch_time))
        train_loss = np.average(train_loss)
        vali_loss, val_accuracy = vali(vali_data, vali_loader, criterion)
        test_loss, test_accuracy = vali(test_data, test_loader, criterion)

        print(
            "Epoch: {0}, Steps: {1} | Train Loss: {2:.3f} Vali Loss: {3:.3f} Vali Acc: {4:.3f} Test Loss: {5:.3f} Test Acc: {6:.3f}"
            .format(epoch + 1, train_steps, train_loss, vali_loss, val_accuracy, test_loss, test_accuracy))
        early_stopping(-val_accuracy, model, path)
        if early_stopping.early_stop:
            print("Early stopping")
            break
        if (epoch + 1) % 5 == 0:
            adjust_learning_rate(model_optim, epoch + 1, args)
def test():
    test_data, test_loader = get_data(flag='TRAIN')

    preds = []
    trues = []
    folder_path = './test_results/' + setting + '/'
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    model.eval()
    with torch.no_grad():
        for i, (batch_x, label, padding_mask) in enumerate(test_loader):
            batch_x = batch_x.float().to(device)
            padding_mask = padding_mask.float().to(device)
            label = label.to(device)

            outputs = model(batch_x, padding_mask, None, None)

            preds.append(outputs.detach())
            trues.append(label)

    preds = torch.cat(preds, 0)
    trues = torch.cat(trues, 0)
    print('test shape:', preds.shape, trues.shape)

    probs = torch.nn.functional.softmax(preds)  # (total_samples, num_classes) est. prob. for each class and sample
    predictions = torch.argmax(probs, dim=1).cpu().numpy()  # (total_samples,) int class index for each sample
    trues = trues.flatten().cpu().numpy()
    accuracy = cal_accuracy(predictions, trues)
    print('test accuracy:', accuracy)

In [43]:
train()
test()

40
40
40
Epoch: 1 cost time: 0.7035861015319824
Epoch: 1, Steps: 3 | Train Loss: 1.935 Vali Loss: 0.695 Vali Acc: 0.750 Test Loss: 0.630 Test Acc: 0.725
Validation loss decreased (inf --> -0.750000).  Saving model ...
Epoch: 2 cost time: 0.6311109066009521
Epoch: 2, Steps: 3 | Train Loss: 0.522 Vali Loss: 0.254 Vali Acc: 0.850 Test Loss: 0.249 Test Acc: 0.875
Validation loss decreased (-0.750000 --> -0.850000).  Saving model ...
Epoch: 3 cost time: 0.7240602970123291
Epoch: 3, Steps: 3 | Train Loss: 0.154 Vali Loss: 0.100 Vali Acc: 0.975 Test Loss: 0.114 Test Acc: 0.975
Validation loss decreased (-0.850000 --> -0.975000).  Saving model ...
Epoch: 4 cost time: 0.7057321071624756
Epoch: 4, Steps: 3 | Train Loss: 0.066 Vali Loss: 0.003 Vali Acc: 1.000 Test Loss: 0.019 Test Acc: 0.975
Validation loss decreased (-0.975000 --> -1.000000).  Saving model ...
Epoch: 5 cost time: 0.6929035186767578
Epoch: 5, Steps: 3 | Train Loss: 0.007 Vali Loss: 0.106 Vali Acc: 0.950 Test Loss: 0.417 Test Acc: